In [1]:
# import packages
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score
from bt_classes import my_backtest, test_indicator
# orig_df = pd.read_csv('../res/test_data.csv',index_col=0,parse_dates=True)
# my_backtest(orig_df[:500])
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
import seaborn as sns
from scipy.stats import pearsonr, spearmanr
rcParams['figure.figsize'] = 20,10
# df = pd.read_csv('../res/input0130.csv')

orig_df = pd.read_csv('../xau_1d_20y.csv')
orig_df['datetime'] = pd.to_datetime(orig_df['date'])
orig_df = orig_df.set_index('datetime')

df = orig_df.copy()
df['log_r'] = np.log(df['close']) - np.log(df['open'])
df['label'] = np.sign(df['log_r'].shift(-1))
df['label'][df['label']==-1] = 0
df['label'].fillna(0,inplace=True)

In [2]:
#importing required libraries
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
# follow the literature
# we don't use min-max scaling here, use partial mean-std scaling instead
from sklearn.preprocessing import StandardScaler
from itertools import chain

# and we define our model here
def lstm_model(sample_len=120,para_a=42, para_b=17):
    model = Sequential()
    model.add(LSTM(units=para_a, dropout=0.1, return_sequences=True, input_shape=(sample_len,1),activation='tanh'))# (25,15)-57, (42,17)-58
    model.add(LSTM(units=para_b, dropout=0.08, activation='tanh'))
    # model.add(Dropout(0.08))# 加了之后同原先效果差不多，（应该一定程度上）可以防止过拟合
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

Using TensorFlow backend.


In [3]:
sample_len = 120
train_begin = sample_len
train_end = train_begin + 2000
scaler = StandardScaler()
train_set = df[['log_r','label']][train_begin-sample_len:train_end].reset_index()
x_train, y_train = [], []
x_train_set = list(chain.from_iterable(scaler.fit_transform(train_set['log_r'].values.reshape(-1,1))))
for i in range(sample_len,len(x_train_set)):
    x_train.append(x_train_set[i-sample_len:i])
    y_train.append(train_set['label'][i])
x_train, y_train = np.array(x_train), np.array(y_train)
y_train = to_categorical(y_train,2)
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1)) 

model = lstm_model(sample_len=sample_len)
model.fit(x_train,y_train,epochs=30, batch_size=100, callbacks=[EarlyStopping(monitor='loss',patience=10)])


Epoch 1/30
2000/2000 [==============================] - 5s 2ms/step - loss: 0.6923 - accuracy: 0.5330
Epoch 2/30
2000/2000 [==============================] - 4s 2ms/step - loss: 0.6913 - accuracy: 0.5290
Epoch 3/30
2000/2000 [==============================] - 4s 2ms/step - loss: 0.6911 - accuracy: 0.5240
Epoch 4/30
2000/2000 [==============================] - 4s 2ms/step - loss: 0.6911 - accuracy: 0.5240
Epoch 5/30
2000/2000 [==============================] - 4s 2ms/step - loss: 0.6911 - accuracy: 0.5295
Epoch 6/30
2000/2000 [==============================] - 4s 2ms/step - loss: 0.6906 - accuracy: 0.5305
Epoch 7/30
2000/2000 [==============================] - 4s 2ms/step - loss: 0.6903 - accuracy: 0.5305
Epoch 8/30
2000/2000 [==============================] - 4s 2ms/step - loss: 0.6906 - accuracy: 0.5260
Epoch 9/30
2000/2000 [==============================] - 4s 2ms/step - loss: 0.6905 - accuracy: 0.5240
Epoch 10/30
2000/2000 [==============================] - 4s 2ms/step - loss: 0.69

In [4]:
model.save('d120-2120.h5')  # d日频，240-2240天

In [4]:
from keras.models import load_model
model = load_model('d240-2240_120.h5')  

In [5]:
from tqdm import tqdm
from sa import *
from utils import *

# Please select the last activation layer.
layer_names = ['lstm_2']

default_upper_bound = 2000
default_n_bucket = 1000
default_n_classes = 2
class Args(): #创建一个类
    def __init__(self): #定义初始化信息。
        self.is_classification = True
        self.save_path = './tmp/'
        self.d = 'lstm_r'
        self.num_classes = 2
        self.lsa = True
        self.dsa = True
        self.target = 'none'
        self.batch_size = 128
        self.var_threshold = 1e-5
        self.upper_bound = 2000
        self.n_bucket = 1000
        self.is_classification = True
args = Args()

In [ ]:
import numpy as np
 
def cross_entropy(y_true, y_pred):
    
    Y = np.float_(Y)
    P = np.float_(P)
    return -np.sum(Y * np.log(P) + (1 - Y) * np.log(1 - P))

In [6]:
test_len = 250
step = 25
starter = range(2120,len(df)-test_len,step)
all_results = []
layer_names = ['lstm_2']
for test_begin in starter:
    test_end = test_begin + test_len

    x_test, y_test = [], []
    test_set = df[['log_r','label']][test_begin-250:test_end].reset_index()
    y_true = df['label'][test_begin:test_end]
    x_test_set = list(chain.from_iterable(scaler.transform(test_set['log_r'].values.reshape(-1,1))))
    for i in range(sample_len,len(x_test_set)):
        x_test.append(x_test_set[i-sample_len:i])
        y_test.append(test_set['label'][i])
    x_test, y_test = np.array(x_test), np.array(y_test)
    x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1)) 
    y_test = to_categorical(y_test,2)

    y_pred = model.predict_classes(x_test)

    ti = test_indicator(y_true,y_pred)
    # to_save = pd.DataFrame()
    # to_save['y_true'] = y_true
    # to_save['y_pred'] = y_pred
    # to_save.to_csv(f'{test_begin}_{test_end}_test.csv')
    test_dsa = fetch_dsa(model, x_train, x_test, 'dsa from{}'.format(test_begin), layer_names, args)
    dsa_cov = get_sc(np.amin(test_dsa), 5, 20, test_dsa)
    test_lsa = fetch_lsa(model, x_train, x_test, 'lsa from{}'.format(test_begin), layer_names, args)
    lsa_cov = get_sc(np.amin(test_lsa), 1500, 20, test_lsa)
    eva = model.evaluate(x_test, y_test)
    mark = pd.DataFrame()
    mark['y_true'] = y_true
    mark['y_pred'] = y_pred
    mark['win'] = 0
    mark['win'].loc[mark['y_true']==mark['y_pred']] = 1
    mark['lsa'] = test_lsa
    mark['dsa'] = test_dsa
    lsa_win_pearson = mark[['win','lsa']].corr().iloc[0,1]
    lsa_win_spearman = mark[['win','lsa']].corr(method='spearman').iloc[0,1]
    dsa_win_pearson = mark[['win','dsa']].corr().iloc[0,1]
    dsa_win_spearman = mark[['win','dsa']].corr(method='spearman').iloc[0,1]
    print('Loss: {:.2f}, Accuracy: {:.2f}'.format(eva[0], eva[1]))
    print('Dsa Mean: {:.2f}, Var: {:.2f}, Max: {:.2f}, Min: {:.2f}'.format(np.mean(test_dsa),np.std(test_dsa),np.max(test_dsa),np.min(test_dsa)))
    print('Lsa Mean: {:.2f}, Var: {:.2f}, Max: {:.2f}, Min: {:.2f}'.format(np.mean(test_lsa),np.std(test_lsa),np.max(test_lsa),np.min(test_lsa)))
    test_df = orig_df[test_begin:test_end]
    test_df['label'] = model.predict_classes(x_test)
    sharpe,dd,ar = my_backtest(test_df)
    this_result = [test_begin,test_end,eva[0],eva[1],ti.get_precision(),ti.get_recall(),ti.get_successive_n_failure_rate(),dsa_cov,np.mean(test_dsa),np.std(test_dsa),np.max(test_dsa),np.min(test_dsa),lsa_cov,np.mean(test_lsa),np.std(test_lsa),np.max(test_lsa),np.min(test_lsa),sharpe,dd,ar,lsa_win_pearson,dsa_win_pearson,lsa_win_spearman,dsa_win_spearman]
    all_results.append(this_result)
    rs = pd.DataFrame(all_results,columns=['test_begin','test_end','loss','accuracy','precision','recall','snfr','dsa_cov','dsa_mean','dsa_std','dsa_max','dsa_min','lsa_cov','lsa_mean','lsa_std','lsa_max','lsa_min','sharpe','drawdown','annual','lsa_win_pearson','dsa_win_pearson','lsa_win_spearman','dsa_win_spearman'])
    rs.to_csv('new_rs.csv')

AssertionError: Length of y is 130 while y_pred 250